In [ ]:
import torch, torchvision  
from PIL import Image

# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries like numpy, json, cv2 etc.
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
# create the detectron2 model
cfg = get_cfg()   # get a fresh new config
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = "cpu"
predictor = DefaultPredictor(cfg)

In [ ]:
# load in the keypoint to yoga pose classification model

model = torchvision.models.resnet34()
model.fc = torch.nn.Linear(model.fc.in_features, 70)
saved_state_dict = torch.load("trained_model_state_dict.ckpt")
model.load_state_dict(saved_state_dict)
model.eval()

In [ ]:
id_to_pose = {
    0: 'Boat_Pose_or_Paripurna_Navasana_out',
    1: 'Bound_Angle_Pose_or_Baddha_Konasana_out',
    2: 'Bow_Pose_or_Dhanurasana_out',
    3: 'Bridge_Pose_or_Setu_Bandha_Sarvangasana_out',
    4: 'Camel_Pose_or_Ustrasana_out',
    5: 'Cat_Cow_Pose_or_Marjaryasana_out',
    6: 'Chair_Pose_or_Utkatasana_out',
    7: 'Child_Pose_or_Balasana_out',
    8: 'Cobra_Pose_or_Bhujangasana_out',
    9: 'Cockerel_Poseout',
    10: 'Corpse_Pose_or_Savasana_out',
    11: 'Cow_Face_Pose_or_Gomukhasana_out',
    12: 'Crane_(Crow)_Pose_or_Bakasana_out',
    13: 'Downward-Facing_Dog_pose_or_Adho_Mukha_Svanasana_out',
    14: 'Eagle_Pose_or_Garudasana_out',
    15: 'Eight-Angle_Pose_or_Astavakrasana_out',
    16: 'Extended_Revolved_Side_Angle_Pose_or_Utthita_Parsvakonasana_out',
    17: 'Extended_Revolved_Triangle_Pose_or_Utthita_Trikonasana_out',
    18: 'Feathered_Peacock_Pose_or_Pincha_Mayurasana_out',
    19: 'Firefly_Pose_or_Tittibhasana_out',
    20: 'Fish_Pose_or_Matsyasana_out',
    21: 'Four-Limbed_Staff_Pose_or_Chaturanga_Dandasana_out',
    22: 'Frog_Pose_or_Bhekasanaout',
    23: 'Garland_Pose_or_Malasana_out',
    24: 'Gate_Pose_or_Parighasana_out',
    25: 'Half_Lord_of_the_Fishes_Pose_or_Ardha_Matsyendrasana_out',
    26: 'Half_Moon_Pose_or_Ardha_Chandrasana_out',
    27: 'Handstand_pose_or_Adho_Mukha_Vrksasana_out',
    28: 'Happy_Baby_Pose_or_Ananda_Balasana_out',
    29: 'Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana_out',
    30: 'Heron_Pose_or_Krounchasana_out',
    31: 'Intense_Side_Stretch_Pose_or_Parsvottanasana_out',
    32: 'Legs-Up-the-Wall_Pose_or_Viparita_Karani_out',
    33: 'Locust_Pose_or_Salabhasana_out',
    34: 'Lord_of_the_Dance_Pose_or_Natarajasana_out',
    35: 'Low_Lunge_pose_or_Anjaneyasana_out',
    36: 'Peacock_Pose_or_Mayurasana_out',
    37: 'Plow_Pose_or_Halasana_out',
    38: 'Pose_Dedicated_to_the_Sage_Koundinya_or_Eka_Pada_Koundinyanasana_I_and_IIout',
    39: 'Rajakapotasanaout',
    40: 'Reclining_Hand-to-Big-Toe_Pose_or_Supta_Padangusthasana_out',
    41: 'Revolved_Head-to-Knee_Pose_or_Parivrtta_Janu_Sirsasana_out',
    42: 'Scale_Pose_or_Tolasana_out',
    43: 'Scorpion_pose_or_vrischikasanaout',
    44: 'Seated_Forward_Bend_pose_or_Paschimottanasana_out',
    45: 'Shoulder-Pressing_Pose_or_Bhujapidasana_out',
    46: 'Side_Crane_(Crow)_Pose_or_Parsva_Bakasana_out',
    47: 'Side_Plank_Pose_or_Vasisthasana_out',
    48: 'Sitting pose 1 (normal)out',
    49: 'Split poseout',
    50: 'Standing_Forward_Bend_pose_or_Uttanasana_out',
    51: 'Standing_Split_pose_or_Urdhva_Prasarita_Eka_Padasana_out',
    52: 'Standing_big_toe_hold_pose_or_Utthita_Padangusthasanaout',
    53: 'Supported_Headstand_pose_or_Salamba_Sirsasana_out',
    54: 'Supported_Shoulderstand_pose_or_Salamba_Sarvangasana_out',
    55: 'Supta_Baddha_Konasana_out',
    56: 'Supta_Virasana_Vajrasanaout',
    57: 'Tortoise_Poseout',
    58: 'Tree_Pose_or_Vrksasana_out',
    59: 'Upward_Bow_(Wheel)_Pose_or_Urdhva_Dhanurasana_out',
    60: 'Upward_Plank_Pose_or_Purvottanasana_out',
    61: 'Virasana_or_Vajrasanaout',
    62: 'Warrior_III_Pose_or_Virabhadrasana_III_out',
    63: 'Warrior_II_Pose_or_Virabhadrasana_II_out',
    64: 'Warrior_I_Pose_or_Virabhadrasana_I_out',
    65: 'Wide-Angle_Seated_Forward_Bend_pose_or_Upavistha_Konasana_out',
    66: 'Wide-Legged_Forward_Bend_pose_or_Prasarita_Padottanasana_out',
    67: 'Wild_Thing_pose_or_Camatkarasana_out',
    68: 'Wind_Relieving_pose_or_Pawanmuktasanaout',
    69: 'viparita_virabhadrasana_or_reverse_warrior_poseout'
}

In [ ]:
# image transforms
transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(
            size=(
                224,
                224,
            )
        ),
        torchvision.transforms.ToTensor(),
    ]
)

In [ ]:
# read in the image
img_path = "Images/Boat_Pose_or_Paripurna_Navasana_/275.jpg"
im = cv2.imread(img_path)
Image.fromarray(im[:,:,::-1])

In [ ]:
# run the keypoint detection model
outputs = predictor(im)
v = Visualizer(im[:,:,::-1] * 0, MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

In [ ]:
# visualize the keypoint image
Image.fromarray(out.get_image())

In [ ]:
transformed_img = transform(Image.fromarray(out.get_image()))

In [ ]:
out_model = model(transformed_img.unsqueeze(0))

In [ ]:
print("Yoga class:", id_to_pose[out_model[0].argmax().item()])